In [5]:
from svmutil import *
import pandas as pd
import numpy as np

In [2]:
# Read Preprocessed data
y_train, X_train = svm_read_problem(r'train.data')
y_test, X_test = svm_read_problem(r'test.data')

In [3]:
# Train & Test on different kernel
print('Linear SVM:')
linear_model =  svm_train(y_train, X_train, '-t 0' )
p_label, p_acc, p_val = svm_predict(y_test,  X_test, linear_model)

print('\nPolynoimal SVM:')
poly_model =  svm_train(y_train, X_train, '-t 1' )
p_label, p_acc, p_val = svm_predict(y_test,  X_test, poly_model)

print('\nRBF SVM:')
RBF_model =  svm_train(y_train, X_train, '-t 2' )
p_label, p_acc, p_val = svm_predict(y_test,  X_test, RBF_model)

Linear SVM:
Accuracy = 95.08% (2377/2500) (classification)

Polynoimal SVM:


KeyboardInterrupt: 

In [6]:
def grid_search(y_train, X_train, args='-t 2 -v 5 -q', para_range= {'c':[1,2,3], 'g':[None], 'd':[None], 'r':[None]} ):
    best_acc = 0.0
    best_para = {'c':0, 'g':0, 'd':0, 'r':0}
    for c in para_range['c']:
        for g in para_range['g']:
            for d in para_range['d']:
                for r in para_range['r']:
                    arg_c = '' if c is None else ' -c %s '%str(c)
                    arg_g = '' if g is None else ' -g %s '%str(g)
                    arg_d = '' if d is None else ' -d %s '%str(d)
                    arg_r = '' if r is None else ' -r %s '%str(r)
                    args_ = args + arg_c + arg_g + arg_d + arg_r  
                    acc = svm_train(y_train, X_train, args_)
                    if acc > best_acc:
                        best_acc = acc
                        best_para = {'c':c, 'g':g, 'd':d, 'r':r}
    print("Best Parameters ", best_para , 'Accuracy :', best_acc)

In [5]:
# Linear
grid_search(y_train, X_train, args='-t 0 -v 5 ', para_range= {'c':[1,2,3], 'g':[None], 'd':[None], 'r':[None]})
# Poly
grid_search(y_train, X_train, args='-t 1 -v 5', para_range= {'c':[1,2,3], 'g':[1e-3, 1e-2, 1e-1], 'd':[3,2,1], 'r':[0,1]} )
# RBF
grid_search(y_train, X_train, args='-t 2 -v 5', para_range= {'c':[1,2,3,4,5], 'g':[1e-1, 1e-2, 1e-3, 1e-4], 'd':[None], 'r':[None]})

Best Parameters : {'c': 2, 'g': 0, 'd': 0, 'r': 0} Accuracy : 96.64
Best Parameters  {'c': 2, 'g': 0.1, 'd': 2, 'r': 0} Accuracy : 98.14
Best Parameters  {'c': 3, 'g': 0.01, 'd': 0, 'r': 0} Accuracy : 98.57


In [6]:
# use params found in grid search to train & test model
print('Linear SVM:')
linear_model =  svm_train(y_train, X_train, '-t 0 -c 2.0' )
p_label, p_acc, p_val = svm_predict(y_test,  X_test, linear_model)

print('\nPolynoimal SVM:')
poly_model =  svm_train(y_train, X_train, '-t 1 -c 2  -g 0.1  -d 2  -r 0' )
p_label, p_acc, p_val = svm_predict(y_test,  X_test, poly_model)

print('\nRBF SVM:')
RBF_model =  svm_train(y_train, X_train, '-t 2 -c 3.0  -g 0.01' )
p_label, p_acc, p_val = svm_predict(y_test,  X_test, RBF_model)

Linear SVM:
Accuracy = 95% (2375/2500) (classification)

Polynoimal SVM:
Accuracy = 97.68% (2442/2500) (classification)

RBF SVM:
Accuracy = 98.08% (2452/2500) (classification)


In [7]:
from sklearn.metrics.pairwise import rbf_kernel

def create_kernel(x1, x2, lam_1=1, lam_2=1): # train/train or train/test
    # create dense data for x1 (train)
    max_key=np.max([np.max(v.keys()) for v in x1])
    arr=np.zeros( (len(x1), len(max_key) ))
    for row, vec in enumerate(x1):
        for k, v in vec.items():
            arr[row][k-1]=v
    x1 = np.copy(arr)
    
    #create dense data for x2
    max_key=np.max([np.max(v.keys()) for v in x2])
    arr=np.zeros( (len(x2), len(max_key) ))
    for row, vec in enumerate(x2):
        for k, v in vec.items():
            arr[row][k-1]=v
    x2 = np.copy(arr)
    
    #create a linear kernel matrix 
    k_linear = np.zeros( (len(x2), len(x1)) )
    k_linear = np.dot(x2, x1.T)
    
    #create a RBF kernel matrix 
    k_RBF = np.zeros( (len(x2), len(x1)) )
    k_RBF = rbf_kernel(x2, x1, gamma=0.01)
    
    #create kernel matrix 
    k = np.zeros( (len(x2), len(x1)+1) )
    k[:,1:] = lam_1*k_linear + lam_2*k_RBF
    k[:,:1] = np.arange(len(x2))[:,np.newaxis]+1
    
    kernel = [list(row) for row in k]
    return kernel

In [8]:
lam_1_range = [0.2, 0.4, 0.6, 0.8, 1.0]
lam_2_range = [0.2, 0.4, 0.6, 0.8, 1.0]
for lam_1 in lam_1_range:
    for lam_2 in lam_2_range:
        if lam_1 == lam_2 and lam_1!= 1: continue
        print("lam_1:%.1f, lam_2:%.1f"%(lam_1, lam_2), end = ' | ')
        kernel = create_kernel(X_train, X_train, lam_1, lam_2)
        model = svm_train(y_train, kernel, '-t 4 -c 3 -g 0.01')
        kernel = create_kernel(X_train, X_test, lam_1, lam_2)
        p_label, p_acc, p_val = svm_predict(y_test, kernel, model)

lam_1:0.2, lam_2:0.4 | Accuracy = 95.72% (2393/2500) (classification)
lam_1:0.2, lam_2:0.6 | Accuracy = 95.76% (2394/2500) (classification)
lam_1:0.2, lam_2:0.8 | Accuracy = 95.92% (2398/2500) (classification)
lam_1:0.2, lam_2:1.0 | Accuracy = 96% (2400/2500) (classification)
lam_1:0.4, lam_2:0.2 | Accuracy = 95.12% (2378/2500) (classification)
lam_1:0.4, lam_2:0.6 | Accuracy = 95.44% (2386/2500) (classification)
lam_1:0.4, lam_2:0.8 | Accuracy = 95.48% (2387/2500) (classification)
lam_1:0.4, lam_2:1.0 | Accuracy = 95.48% (2387/2500) (classification)
lam_1:0.6, lam_2:0.2 | Accuracy = 95.04% (2376/2500) (classification)
lam_1:0.6, lam_2:0.4 | Accuracy = 95.16% (2379/2500) (classification)
lam_1:0.6, lam_2:0.8 | Accuracy = 95.4% (2385/2500) (classification)
lam_1:0.6, lam_2:1.0 | Accuracy = 95.44% (2386/2500) (classification)
lam_1:0.8, lam_2:0.2 | Accuracy = 95% (2375/2500) (classification)
lam_1:0.8, lam_2:0.4 | Accuracy = 95.12% (2378/2500) (classification)
lam_1:0.8, lam_2:0.6 | Accu